In [1]:
import pandas as pd
import numpy as np
import glob
import os

In [2]:
def format_gene_name (s):
    for i, c in enumerate(s):
        if c.isdigit():
            return s[:i].upper()
    return s

## Concatenating all Ka/Ks data into one dataframe

In [95]:
#Writing pipeline output to one file

path = r'/Users/alexyu/Downloads/biotools/output'
all_files = sorted(glob.glob(path + "/*.kaks"))
all_files = sorted(all_files, key=lambda name: int(name[46:-5]))

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0, sep='\t')
    li.append(df)

data = pd.concat(li, axis=0, ignore_index=True)
outFile = input('Enter output file name: ')
data.to_csv('kaks/' + outFile, sep='\t')

sig_data = data[(data['Ka/Ks'] > 1) & (data['P-Value(Fisher)'] < 0.1)]

Enter output file name: Microbat_MongolianGerbil_stressreg.txt


In [108]:
sig_data.to_csv('temp.txt', sep='\t')

In [107]:
#Reading from KaKs combined file

inFile = input('Enter output file name: ')
data = pd.read_csv(inFile, sep='\t')
data = data.drop(data.columns[0], axis=1)
data.index += 1

sig_data = data[(data['Ka/Ks'] > 1) & (data['P-Value(Fisher)'] < 0.1)]

Enter output file name: kaks/Hyrax_Hedgehog_stress.txt


## Used for Ensembl full protein genome data

In [62]:
#Formatting significant gene data

gene_df = pd.DataFrame(columns=['Protein', 'Symbol', 'Pair Number', 'Hedgehog Protein', 'Platypus Protein','Ka/Ks', 'P-Value'])

for index, row in sig_data.iterrows():
    file = open("/Users/alexyu/Downloads/biotools/HP_OGs/OG{0}.fa".format(index), mode='r')
    text = file.read()
    
    gene_name_start = text.find("gene_symbol:")
    if gene_name_start < 0:
        gene_name = None
    else:
        gene_name = text[gene_name_start + 12 : text.find(" description:", gene_name_start)]
        
    protein_desc_start = text.find("description:")
    if protein_desc_start < 0:
        protein_desc = None
    else:
        end_pos = text.find(" [", protein_desc_start)
        protein_desc = text[protein_desc_start + 12 : end_pos]
        
    hedgehog_gene = text[1 : text.find(".")]
    platypus_gene = text[text.find(">", text.find(".")) + 1 : text.find(".", text.find(">", text.find(".")) + 1)]
        
    gene_df.loc[len(gene_df)] = [protein_desc, gene_name, index, hedgehog_gene, platypus_gene, row['Ka/Ks'], row['P-Value(Fisher)']]
    
gene_df['Pair Number'] = gene_df['Pair Number'].astype('int64')
gene_df.set_index('Pair Number', inplace=True)

gene_df = gene_df.dropna()
gene_df.to_csv('HP_filtered_genes.txt', sep='\t')

## Used for Ensembl BioMart data queries

In [96]:
gene_df = pd.DataFrame(columns=['Protein', 'Symbol', 'Pair Number', 'Microbat Protein', 'Mongolian Gerbil Protein','Ka/Ks', 'P-Value'])

for index, row in sig_data.iterrows():
    
    file = open("/Users/alexyu/Downloads/workDir/Output/OrthologousGroupsFasta/OG{0}.fa".format(index), mode='r')
    text = file.read()
    
    seqs = text.split('>')
    seq1 = seqs[1]
    seq2 = seqs[2]

    desc1 = seq1.split('|')
    gene_name = desc1[4]
    prot_id1 = desc1[0]

    desc2 = seq2.split('|')
    prot_id2 = desc2[0]

    stop = 0
    prot_desc = ''
    if len(desc1[5]) > 0:
        stop = desc1[5].find(' [')
        prot_desc = desc1[5][:stop]
    elif len(desc2[5]) > 0:
        stop = desc2[5].find(' [')
        prot_desc = desc2[5][:stop]
    
    gene_df.loc[len(gene_df)] = [prot_desc, gene_name, index, prot_id1, prot_id2, row['Ka/Ks'], row['P-Value(Fisher)']]

gene_df['Pair Number'] = gene_df['Pair Number'].astype('int64')
gene_df.set_index('Pair Number', inplace=True)

gene_df = gene_df.dropna()
outFile = input('Enter output file name: ')
gene_df.to_csv('filteredGenes/' + outFile, sep='\t')

Enter output file name: Microbat_MongolianGerbil_stressreg.txt


## Analyzing Common Positively-Selected Genes

In [12]:
genefiles = glob.glob(os.path.join('filteredGenes/stress', '*.txt'))

# loop through the files and read them in with pandas
dataframes = []
for csvfile in genefiles:
    df = pd.read_csv(csvfile, sep='\t')
    dataframes.append(df)

# concatenate them all together
result = pd.concat(dataframes, ignore_index=True, sort=False)
result = result[['Protein', 'Symbol']]
result = result.dropna()
result['Gene Prefix'] = result['Symbol'].map(format_gene_name)
result['Symbol'] = result['Symbol'].str.upper()

result.sort_values(by=['Gene Prefix'])[result.duplicated(['Gene Prefix'], keep='first')]

/Users/alexyu/.pyenv/versions/3.6.0/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


,Protein,Symbol,Gene Prefix
59,autophagy related 7,ATG7,ATG
44,bone morphogenetic protein receptor type 1A,BMPR1A,BMPR
37,chromobox 5,CBX5,CBX
80,growth arrest specific 6,GAS6,GAS
2,GATA,GATA1,GATA
38,interleukin 1 beta,IL1B,IL
89,"NSE3 homolog, SMC5-SMC6 complex component",NSMCE3,NSMCE
90,solute carrier family 30 member 10,SLC30A10,SLC


In [13]:
result.sort_values(by=['Symbol']).drop_duplicates(['Symbol']).reset_index(drop=True) #.to_csv('all_kaks.txt', sep='\t')

,Protein,Symbol,Gene Prefix
0,alkaline ceramidase 2,ACER2,ACER
1,angiopoietin 4,ANGPT4,ANGPT
2,autophagy related 14,ATG14,ATG
3,autophagy related 7,ATG7,ATG
4,beta-2-microglobulin,B2M,B
5,bone morphogenetic protein 2,BMP2,BMP
6,bone morphogenetic protein receptor type 1A,BMPR1A,BMPR
7,bone morphogenetic protein receptor type 2,BMPR2,BMPR
8,chromobox 1,CBX1,CBX
9,chromobox 5,CBX5,CBX
